Kaggle - python

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d crmercado/tweets-blogs-news-swiftkey-dataset-4million/

In [ ]:
! mkdir kaggle_data
! unzip "/content/tweets-blogs-news-swiftkey-dataset-4million.zip" -d kaggle_data

Pyro Installation

In [ ]:
!pip3 install torchvision
!pip3 install pyro-ppl

In [ ]:
import os
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
import pyro.poutine as poutine
from torch.distributions import constraints
from pyro.nn import PyroModule, PyroParam, PyroSample
from pyro.nn.module import to_pyro_module_
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoNormal
from pyro.optim import Adam

Data Cleaning


In [ ]:
from google.colab import files
from bs4 import BeautifulSoup
import requests
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [ ]:
#stop word analysis
nltk.download('stopwords')
# Create a set of stop words
stop_words = set(stopwords.words('english'))

# Define a function to remove stop words from a sentence
def remove_stop_words(sentence):
  # Split the sentence into individual words
  words = sentence.split()

  # Use a list comprehension to remove stop words
  filtered_words = [word for word in words if word not in stop_words]

  # Join the filtered words back into a sentence
  return ' '.join(filtered_words)

In [ ]:
import re

file=open("/content/kaggle_data/final/en_US/en_US.news.txt","r")
file.seek(0)
l1 = file.readlines()
print(l1[0])
res = []

#only 10000 paras allowed
for para in l1[:1000]:
  temp = para
  #temp = remove_stop_words(para)
  temp = temp.lower()
  res = res + [x for x in re.split("[//.|//!|//?]", temp) if x!=""]
print(res)
text_data = res
#setnences made above

# Remove special characters and words between them using regex
text_data = [re.sub(r"\[.*?\]", "", text) for text in text_data]

# Remove words not in the English alphabet
english_alphabet = set(string.ascii_lowercase)
text_data = [' '.join([word for word in text.split() if all(char in english_alphabet for char in word)]) for text in text_data]

# Remove leading/trailing whitespaces
text_data =  text_data

# Remove empty sentences
# text_data =

# Create a DataFrame with the cleaned text data
df = pd.DataFrame({"Text": text_data})

# Save the cleaned text data to a CSV file
output_path = "output.csv"
# Set index=False to exclude the index column in the output
df.to_csv(output_path, index=False)
print (len(df))
print("Text data cleaned and saved to:", output_path)

Classes


In [ ]:

from torch.utils.data import DataLoader, random_split
from collections import Counter
from torch import nn
import torch

In [ ]:

#class TextDataset(torch.utils.data.Dataset):
class TextDataset(torch.utils.data.Dataset):
	def __init__(self, args):
		self.args = args
		self.words = self.load_words()
		self.unique_words = self.get_unique_words()

		self.index_to_word = {index: word for index, word in enumerate(self.unique_words)}
		self.word_to_index = {word: index for index, word in enumerate(self.unique_words)}

		self.word_indexes = [self.word_to_index[w] for w in self.words]

	def load_words(self):
		train_df = pd.read_csv('/content/output.csv')
		text = train_df['Text'].str.cat(sep=' ')
		return text.split(' ')

	def get_unique_words(self):
		word_counts = Counter(self.words)
		return sorted(word_counts, key=word_counts.get, reverse=True)

	def __len__(self):
		return len(self.word_indexes) - self.args

	def __getitem__(self, index):
		return (
			torch.tensor(self.word_indexes[index:index + self.args]),
			torch.tensor(self.word_indexes[index + 1:index + self.args+ 1])
		)

In [ ]:

from torch import nn
class LSTMModel(nn.Module):
	def __init__(self, dataset):
		super(LSTMModel, self).__init__()
		self.lstm_size = 128
		self.embedding_dim = 128
		self.num_layers = 3
		self.n = True

		n_vocab = len(dataset.unique_words)
		self.embedding = nn.Embedding(
			num_embeddings=n_vocab,
			embedding_dim=self.embedding_dim,
		)
		self.lstm = nn.LSTM(
			input_size=self.embedding_dim,
			hidden_size=self.lstm_size,
			num_layers=self.num_layers,
			dropout=0.2,
		)
		self.fc = nn.Linear(self.lstm_size, n_vocab)

	def forward(self, x=None, prev_state=None):
		#if (self.n == True):
			# print(prev_state)
			# print("----")
			# print(model)
		global saving_state2
		saving_state2 = prev_state
		embed = self.embedding(x)
		self.n = False
		output, state = self.lstm(embed, prev_state)
		#print(prev_state)
		logits = self.fc(output)

		return logits, state

	def init_state(self, sequence_length):
		return (
			torch.zeros(self.num_layers, sequence_length, self.lstm_size), torch.zeros(self.num_layers, sequence_length, self.lstm_size)
		)

In [ ]:
from torch.utils.data import DataLoader, random_split
from collections import Counter
import copy
saving_state2 = 1
# Hyperparameters
sequence_length = 10
batch_size = 64
learning_rate = 0.003
num_epochs = 20

# Create the dataset
dataset = TextDataset(sequence_length)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset,
								[train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset,
					batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset,
						batch_size=batch_size)

# Create the model
model = LSTMModel(dataset)
pyro_model = copy.deepcopy(model)
#to_pyro_module_(pyro_model)
#to_pyro_module_(model)
# print(model)
# print(pyro_model)
# print("----")
# # Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
	model.train()
	total_loss = 0.0

	for batch in train_loader:
		inputs, targets = batch

		optimizer.zero_grad()

		hidden = model.init_state(sequence_length)
		outputs, _ = model(inputs, hidden)

		loss = criterion(outputs.view(-1,
					len(dataset.unique_words)), targets.view(-1))
		loss.backward()

		optimizer.step()

		total_loss += loss.item()

	# Calculate average loss for the epoch
	average_loss = total_loss / len(train_loader)

	# Print the epoch and average loss
	print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}")

	# Validation loop
	model.eval()
	val_loss = 0.0

	with torch.no_grad():
		for batch in val_loader:
			inputs, targets = batch

			hidden = model.init_state(sequence_length)
			outputs, _ = model(inputs, hidden)

			loss = criterion(outputs.view(-1,
							len(dataset.unique_words)), targets.view(-1))
			val_loss += loss.item()

	# Calculate average validation loss for the epoch
	average_val_loss = val_loss / len(val_loader)

	# Print the epoch and average validation loss
	print(f"Epoch[{epoch+1}/{num_epochs}], Validation Loss: {average_val_loss: .4f}")


In [ ]:
#pyro execution
from torch.utils.data import DataLoader, random_split
from collections import Counter
import copy
import pyro.infer.autoguide as autoguide
from pyro.infer import SVI, TraceGraph_ELBO
import torch.optim as optim
sequence_length = 10
batch_size = 64
learning_rate = 0.003
num_epochs = 20

# Create the dataset
dataset = TextDataset(sequence_length)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset,
								[train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset,
					batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset,
						batch_size=batch_size)

# Create the model
model = LSTMModel(dataset)
to_pyro_module_(model)
guide = autoguide.AutoNormal(model)
optimizer = Adam({"lr": 0.01})
#optimizer = torch.optim.SGD
scheduler = pyro.optim.ExponentialLR({'optimizer': optimizer, 'optim_args': {'lr': 0.01}, 'gamma': 0.1})
svi = SVI(model, guide, scheduler, loss=TraceGraph_ELBO())

for epoch in range(num_epochs):
	model.train()
	total_loss = 0.0

	for batch in train_loader:
		inputs, targets = batch

		hidden = model.init_state(sequence_length)
		outputs, _ = model(inputs, hidden)

		loss = criterion(outputs.view(-1,
					len(dataset.unique_words)), targets.view(-1))
		loss.backward()

		#optimizer.step()

		total_loss += loss.item()

	# Calculate average loss for the epoch
	average_loss = total_loss / len(train_loader)

	# Print the epoch and average loss
	print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}")

	# Validation loop
	model.eval()
	val_loss = 0.0
	scheduler.step()


In [ ]:
# Input a sentence
list_setences = ["his family is", "on his way to","in the end he"]

# Preprocess the input sentence
for input_sentence in list_setences:
  print(input_sentence)
  input_indexes = [dataset.word_to_index[word] for word in input_sentence.split()]
  input_tensor = torch.tensor(input_indexes, dtype=torch.long).unsqueeze(0)

  # Generate the next word
  model.eval()
  hidden = model.init_state(len(input_indexes))
  #print(hidden)
  outputs, _ = model(input_tensor, hidden)
  predicted_index = torch.argmax(outputs[0, -1, :]).item()
  predicted_word = dataset.index_to_word[predicted_index]

  # Print the predicted word
  print("Input Sentence:", input_sentence)
  print("Predicted Next Word python:", predicted_word)